In [2]:
import pandas as pd 
import numpy as np
import matplotlib as  plt
import glob

In [3]:
# 处理铝20天
label_20 = pd.read_csv('/home/wph/course/data_science/Project_FinIR/Train_data/Label_LMEAluminium_train_20d.csv', names=['index','date','label'],header=0,index_col='date').drop('index',axis=1)
lv = pd.read_csv('/home/wph/course/data_science/Project_FinIR/Train_data/LMEAluminium3M_train.csv',names=['index','date','open','high','low','close','volume'], header=0,index_col='date').drop('index', axis=1)
lv_oi = pd.read_csv('/home/wph/course/data_science/Project_FinIR/Train_data/LMEAluminium_OI_train.csv', names=['index','date','oi'],header=0,index_col='date').drop('index',axis=1)

feature_list = [lv, lv_oi]

indices_path = glob.glob(r'Train_data/Indices*')
indices_path.sort()
indices_name = ['dxy','nky', 'shsz','spx','sx5e','ukx','vix']
for i, path in enumerate(indices_path):
    feature_list.append(pd.read_csv(path, names=['index','date',indices_name[i]], header=0, index_col='date').drop('index',axis=1))

train = label_20.join(feature_list, how='left')
train

,label,open,high,low,close,volume,oi
date,,,,,,,
2003-01-02,1.0,1337.0,1356.00,1335.0,1353.5,NaN,NaN
2003-01-03,1.0,1359.0,1360.00,1345.0,1348.0,NaN,NaN
2003-01-06,1.0,1348.0,1350.00,1340.0,1349.0,NaN,NaN
2003-01-07,1.0,1341.0,1351.00,1341.0,1348.5,NaN,NaN
2003-01-08,1.0,1347.0,1360.00,1346.0,1357.0,NaN,NaN
...,...,...,...,...,...,...,...
2017-12-21,1.0,2122.0,2151.00,2113.5,2148.0,67302.0,113508.0
2017-12-22,1.0,2148.0,2204.00,2130.0,2192.0,58764.0,6750.0
2017-12-27,0.0,2190.0,2254.00,2170.0,2252.0,71699.0,6238.0


In [4]:
# missing value
cols = train.select_dtypes('number')
print('Missing ratio:')
for col in cols:
    print('{}:{}'.format(col, train[col].isna().sum()/len(train)))
    med = train[col].median()
    train[col].fillna(med,inplace=True)

target = train['label'].values
data = train.drop('label',axis=1).values

Missing ratio:
label:0.0
open:0.0
high:0.0
low:0.0
close:0.0
volume:0.06411609498680738
oi:0.10158311345646438


In [5]:
# 标准化
# from sklearn.preprocessing import scale

# data = scale(data)
# data

In [6]:
# 将10个连续的数据拼接
# concat_data = data
# window_size=10
# for i in range(window_size):
#     data_shift = data.shift(i+1, fill_value=0)
#     concat_data = pd.concat([concat_data, data_shift], axis=1)
# concat_data

In [7]:
import sklearn
import lightgbm as lgb

from sklearn.model_selection import train_test_split

train_size = int(len(data)*0.8)
X_valid = data[train_size:]
Y_valid = target[train_size:]
X_train = data[:train_size]
Y_train = target[:train_size]
# 打乱顺序
X_train = np.random.shuffle(X_train)
Y_train = np.random.shuffle(Y_train)


D_train = lgb.Dataset(X_train, Y_train)
D_valid = lgb.Dataset(X_valid, Y_valid)

In [8]:
# LightGBM
params = dict(boosting_type='gbdt',
            objective='binary',
            # max_depth=10,
            # num_leaves=10**2-1,
            learning_rate=0.001,
            min_child_weight=1,
            # colsample_bytree=0.8,
            # colsample_bynode=0.8,
            reg_alpha=0,
            reg_lambda=0.1,
            n_jobs=-1,
            random_state=2020,
            device='gpu',
            gpu_platform_id=0,
            gpu_device_id=0,
            #   first_metric_only=True,
            metric=['binary_error']
            )
eval_results = dict()
num_round = 5000
valid_sets = [D_train, D_valid]
valid_names = ['train', 'valid']
lgb_reg = lgb.train(params,
                    D_train,
                    num_boost_round=num_round,
                    valid_sets=valid_sets,
                    valid_names=valid_names,
                    early_stopping_rounds=50,
                    evals_result=eval_results,
                    verbose_eval=20
                    # early_stopping_rounds=50
                    )

In [12]:
# 获取数据，按照给定的窗口，每个样本由当前七天的样本构成。
window = 32
X_train, Y_train=[],[]
for i in range(window):
    padding = np.zeros((window-1-i, data.shape[1]),dtype=float)
    tmp = data[:i+1]
    tmp = np.vstack((padding, tmp))
    assert tmp.shape == (window, data.shape[1])
    X_train.append(tmp)
    Y_train.append(target[i])
for i in range(window, len(train)):
    X_train.append(data[i-window:i])
    Y_train.append(target[i])

X_train = np.array(X_train)
Y_train = np.array(Y_train).astype(float)
Y_train = np.expand_dims(Y_train,-1)
print(X_train.shape)
print(Y_train.shape)


(3790, 32, 13)
(3790, 1)


In [14]:
# 使用tcn
import tensorflow as tf 
from tensorflow import keras
from tcn import TCN, tcn_full_summary
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input, Model

gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')


batch_size , timesteps, input_dim = None, window, X_train.shape[2]

i = Input(batch_shape=(batch_size, timesteps, input_dim))

o = TCN(dilations=[1,2,4,8,16], return_sequences=False, nb_filters=16, dropout_rate=0.1)(i)
o = Dense(1, activation='sigmoid')(o)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', factor=0.7, patience=10, verbose=10, mode='auto',
    min_delta=0.0001, cooldown=0, min_lr=0.00005
)

optimizer=keras.optimizers.RMSprop(learning_rate=0.003)	

m = Model(inputs=[i], outputs=[o])
m.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
tcn_full_summary(m, expand_residual_blocks=False)

m.fit(X_train,Y_train, epochs=300, batch_size=64, validation_split=0.2, shuffle=True, callbacks=[reduce_lr])

.5544 - val_loss: 5.0873 - val_accuracy: 0.5409 - lr: 2.8243e-04
Epoch 189/300
46/48 [===========================>..] - ETA: 0s - loss: 2.6922 - accuracy: 0.5435
Epoch 00189: ReduceLROnPlateau reducing learning rate to 0.00025418660952709616.
48/48 [==============================] - 0s 10ms/step - loss: 2.6895 - accuracy: 0.5432 - val_loss: 8.0578 - val_accuracy: 0.4538 - lr: 2.8243e-04
Epoch 190/300
48/48 [==============================] - 1s 11ms/step - loss: 4.5471 - accuracy: 0.5310 - val_loss: 6.9846 - val_accuracy: 0.5369 - lr: 2.5419e-04
Epoch 191/300
48/48 [==============================] - 0s 9ms/step - loss: 2.5120 - accuracy: 0.5587 - val_loss: 3.3578 - val_accuracy: 0.5264 - lr: 2.5419e-04
Epoch 192/300
48/48 [==============================] - 0s 9ms/step - loss: 2.1894 - accuracy: 0.5534 - val_loss: 8.9806 - val_accuracy: 0.5422 - lr: 2.5419e-04
Epoch 193/300
48/48 [==============================] - 0s 10ms/step - loss: 3.3770 - accuracy: 0.5402 - val_loss: 3.3170 - val_ac

In [1]:
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input, Model
import tensorflow as tf
from tcn import TCN, tcn_full_summary

print(len(tf.config.experimental.list_physical_devices('GPU')))

batch_size, timesteps, input_dim = None, 20, 1


def get_x_y(size=1000):
    import numpy as np
    pos_indices = np.random.choice(size, size=int(size // 2), replace=False)
    x_train = np.zeros(shape=(size, timesteps, 1))
    y_train = np.zeros(shape=(size, 1))
    x_train[pos_indices, 0] = 1.0
    y_train[pos_indices, 0] = 1.0
    return x_train, y_train

x, y = get_x_y()
print(x.shape)
print(y.shape)
i = Input(batch_shape=(batch_size, timesteps, input_dim))

o = TCN(return_sequences=False)(i)  # The TCN layers are here.
o = Dense(1)(o)

m = Model(inputs=[i], outputs=[o])
m.compile(optimizer='adam', loss='mse')

tcn_full_summary(m, expand_residual_blocks=False)

m.fit(x, y, epochs=10, validation_split=0.2)

4
(1000, 20, 1)
(1000, 1)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20, 1)]           0         
_________________________________________________________________
residual_block_0 (ResidualBl [(None, 20, 64), (None, 2 8576      
_________________________________________________________________
residual_block_1 (ResidualBl [(None, 20, 64), (None, 2 16512     
_________________________________________________________________
residual_block_2 (ResidualBl [(None, 20, 64), (None, 2 16512     
_________________________________________________________________
residual_block_3 (ResidualBl [(None, 20, 64), (None, 2 16512     
_________________________________________________________________
residual_block_4 (ResidualBl [(None, 20, 64), (None, 2 16512     
_________________________________________________________________
residual_block_5 (ResidualBl [(None

In [1]:
# 测试rolling
import pandas as pd 
df = pd.DataFrame({'a':list(range(12)), 'b':list(range(1,13))})
df

,a,b
0,0,1
1,1,2
2,2,3
3,3,4
4,4,5
5,5,6
6,6,7
7,7,8
8,8,9
9,9,10


In [8]:
df.rolling(3,min_periods=1)
for a,b in df.rolling(3):
    print(a)
    print(b)

NotImplementedError: See issue #11704 https://github.com/pandas-dev/pandas/issues/11704